<a href="https://colab.research.google.com/github/Danzigerrr/PyTorch-Practice/blob/Neural-Network-Model-1/Neural_Network_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Neural Network model

Data - IRIS dataset

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Create Model Class

In [5]:
class Model(nn.Module):
  def __init__(self, input_features, hidden_1, hidden_2, out_features):
    super().__init__()
    self.fc1 = nn.Linear(input_features, hidden_1)
    self.fc2 = nn.Linear(hidden_1, hidden_2)
    self.out = nn.Linear(hidden_2, out_features)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x



## Create a Model instance

In [9]:
torch.manual_seed(37)

input_features = 4
hidden_1 = 8
hidden_2 = 9
out_features = 3

model = Model(input_features, hidden_1, hidden_2, out_features)